In [4]:
from interface import *

In [5]:
## Test code for design 11 (brainfuck chip)
print("{:06b}".format(11))

# instantiate and enable
chip = Chip("/dev/tty.usbmodem2101", 115200)
chip.step_clock(10)

001011


In [6]:
## https://github.com/asinghani/18224-s23-tapeout/blob/main/designs/d11_gbailey_bfchip/dev/src/merged_test.py

IoNone          = 0b000
IoOpcode        = 0b001
IoAddrHi        = 0b010
IoAddrLo        = 0b011
IoReadWrite     = 0b100
BusNone         = 0b000
BusReadProg     = 0b010
BusReadData     = 0b100
BusWriteData    = 0b101
BusReadIo       = 0b110
BusWriteIo      = 0b111

class Inputs:
  def __init__(self, chip):
    self.bus_in = 0
    self.op_done = 0
    self.enable = 0
    self.chip = chip

  def set(self):
    val = (self.enable << 9) | (self.op_done << 8) | self.bus_in
    self.chip.set_all_inputs(val)

  def __repr__(self):
    return f"bus_in={self.bus_in:08b}, " \
           f"op_done={self.op_done:b}, " \
           f"enable={self.enable:b}"

class Outputs:
  def __init__(self, chip):
    self.bus_out = 0
    self.state = 0
    self.halted = 0
    self.chip = chip

  def get(self):
    out = self.chip.get_all_outputs()
    self.bus_out = out & 0x0FF
    self.state = (out & 0x700) >> 8
    self.halted = (out & 0x800) >> 11

  def __repr__(self):
    return f"bus_out={self.bus_out:08b}, " \
           f"state={self.state:03b}, " \
           f"halted={self.halted:b}"

# Modified version of Cristofani's IO test (CC-BY-SA). Includes a loop-forward
# check to make sure we're checking all the states.
prog = bytearray(b"++--[++++++++.]>,>+++++++++,>+++++++++++[<++++++<++++++<+>>>-]<<.>.<<-.>.>.<<.")
stdin = bytearray(b"\n")
input_end = 0
stdout = bytearray(b"LB\nLB\n")

inputs = Inputs(chip)
outputs = Outputs(chip)

print("resetting")
chip.reset_sequence()

inputs.enable = 1
inputs.set()
outputs.get()

# pseudo-registers
opcode = 0
addr_hi = 0
addr_lo = 0

data = bytearray(65536)
output = bytearray()

cycles = 0

while not outputs.halted:
  outputs.get()
  # print(outputs)
  if outputs.state == IoOpcode:
    opcode = outputs.bus_out
  elif outputs.state == IoAddrHi:
    addr_hi = outputs.bus_out
  elif outputs.state == IoAddrLo:
    addr_lo = outputs.bus_out
  elif outputs.state == IoReadWrite:
    addr = (addr_hi << 8) | addr_lo
    if opcode == BusReadProg:
      if addr >= len(prog):
        inputs.bus_in = 0
      else:
        inputs.bus_in = prog[addr]
      print(f"reading program [{addr:04x}]: {chr(inputs.bus_in)}")
    elif opcode == BusReadData:
      inputs.bus_in = data[addr]
      print(f"reading data    [{addr:04x}]: {inputs.bus_in}")
    elif opcode == BusWriteData:
      data[addr] = outputs.bus_out
      print(f"writing data    [{addr:04x}]: {outputs.bus_out}")
    elif opcode == BusReadIo:
      if len(stdin) > 0:
        inputs.bus_in = stdin.pop(0)
      else:
        inputs.bus_in = input_end
      print(f"reading io            : {chr(inputs.bus_in)}")
    elif opcode == BusWriteIo:
      print(f"writing io            : {chr(outputs.bus_out)}")
      output.append(outputs.bus_out)
    inputs.op_done = 1
    inputs.set()
  # print(inputs)
  chip.step_clock()
  cycles += 1

print(f"halted, ran {cycles} cycles")
if output == stdout:
  print(f"output: {bytes(output)}, success!")
else:
  print(f"output: {bytes(output)}, differs from expected: {bytes(stdout)}")
assert(output == stdout)

resetting
reading program [0000]: +
reading data    [0000]: 0
writing data    [0000]: 1
reading program [0001]: +
reading data    [0000]: 1
writing data    [0000]: 2
reading program [0002]: -
reading data    [0000]: 2
writing data    [0000]: 1
reading program [0003]: -
reading data    [0000]: 1
writing data    [0000]: 0
reading program [0004]: [
reading data    [0000]: 0
reading program [0005]: +
reading program [0006]: +
reading program [0007]: +
reading program [0008]: +
reading program [0009]: +
reading program [000a]: +
reading program [000b]: +
reading program [000c]: +
reading program [000d]: .
reading program [000e]: ]
reading program [000f]: >
reading program [0010]: ,
reading io            : 

writing data    [0001]: 10
reading program [0011]: >
reading program [0012]: +
reading data    [0002]: 0
writing data    [0002]: 1
reading program [0013]: +
reading data    [0002]: 1
writing data    [0002]: 2
reading program [0014]: +
reading data    [0002]: 2
writing data    [0002]: 3
r